#### Part 1 Instructions
#### Get the Weather Description and Amount of Precipitation for Each City

In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import gmaps
import requests
import os
import time
import json
import matplotlib.pyplot as plt
from citipy import citipy
from datetime import datetime
# Import the API key.
from config import g_key
from config import weather_api_key
from scipy.stats import linregress


In [2]:
# Create a set of random latitude and longitude combinations 1500.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

616

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# url = "http://api.openweathermap.org/data/2.5/forecast?units=Imperial&APPID=" + weather_api_key

In [7]:
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    # print(f'I:{i}, City:{city}')
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city.
    city_url = f'{url}&q={city.replace(" ","+")}'
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Request city url and pasrde the data
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        try:
            city_amount_of_rainfall_inches = city_weather["rain"]["3h"]
        except KeyError:
            city_amount_of_rainfall_inches = 0
        try:
            city_amount_of_snowfall_inches = city_weather["snow"]["3h"]
        except KeyError:
            city_amount_of_snowfall_inches = 0
        
        # Convert the date to ISO standard.
        # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description,
                          "Rain (inches)" : city_amount_of_rainfall_inches,
                          "Snow (inches)" : city_amount_of_snowfall_inches })

    except:
        print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skip

In [8]:
print(f'Len of city data: {len(city_data)}')

Len of city data: 570


In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Bredasdorp,ZA,-34.53,20.04,55.40,87,0,6.76,clear sky,0,0
1,Hobart,AU,-42.88,147.33,46.00,93,20,5.82,light rain,0,0
2,Muros,ES,42.78,-9.06,62.01,88,45,1.99,scattered clouds,0,0
3,Baruun-Urt,MN,46.68,113.28,68.47,65,99,13.40,overcast clouds,0,0
4,Hilo,US,19.73,-155.09,82.40,58,90,4.70,overcast clouds,0,0
5,Butaritari,KI,3.07,172.79,82.92,72,91,11.32,light rain,0,0
6,Mataura,NZ,-46.19,168.86,39.00,83,99,3.00,overcast clouds,0,0
7,Cape Town,ZA,-33.93,18.42,62.01,87,40,6.93,scattered clouds,0,0
8,Saint George,US,37.10,-113.58,96.01,8,1,8.05,clear sky,0,0
9,Lexington Park,US,38.27,-76.45,73.40,83,75,5.82,light rain,0,0


In [10]:
# Number  cities recorded rainfall
city_data_df.loc[city_data_df["Rain (inches)"] > 0].count()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64

In [11]:
# Number of cities recoded snow fall
city_data_df.loc[city_data_df["Snow (inches)"] > 0].count()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64

In [12]:
# Save the new DataFrame as a CSV file to be used for Part 2.
file_name = os.path.join(".","data","WeatherPy_database.csv")
city_data_df.to_csv(file_name,  index_label="City_ID")
# city_data_df.to_csv(file_name)